In [1]:
'''
author: Felix Hol
date: 2020 Oct 28
content: code to calculate distance of nearest mosquito and identify its particle number
'''

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import itertools as it
import pandas as pd
import pims
import skimage
from skimage import data, io, util
from skimage.measure import label, regionprops
from skimage.morphology import binary_dilation, erosion, dilation, opening, binary_closing, closing, white_tophat, remove_small_objects, disk, black_tophat, skeletonize, convex_hull_image
import scipy
import trackpy as tp
import pylab
import math
from joblib import Parallel, delayed
import multiprocessing
from datetime import datetime
from tqdm import tnrange, tqdm
import pickle
import glob
import cv2 as cv

In [2]:
#### set directories where to get images and where to store output, and specifics of experiment

# dataDir = '/run/user/1001/gvfs/smb-share:server=gaia.pasteur.fr,share=%40ivi/BITES_BLOOD_BEHAVIOR/P3/200923_KPPTN/ctrl02/'
dataDir = '/mnt/DATA/biteData/P3/200930_KPPTN/denv04/'
saveDir = '/home/felix/biteData/P3/200930_processed/'
species = 'aeg'
mosAge = 21      #### mosquito age in days
mosDataName = 'aeg_denv04_0930'
frames = pims.ImageSequence(dataDir+'/*.png')

In [3]:
trackPickleName = saveDir + mosDataName + '_tracks.pkl'
tFull = pd.read_pickle(trackPickleName)

In [4]:
tFull.head()

,y,x,frame,particle,species,age
1,867.825364,1238.666155,500,1,aeg,21
2,1048.280592,1347.122175,500,2,aeg,21
3,2218.832468,1913.488273,500,3,aeg,21
4,863.934177,1238.764141,501,1,aeg,21
5,1048.076491,1347.182610,501,2,aeg,21


In [5]:
T = tFull.copy()
T['nnDist'] = np.nan
T['nnP'] = np.nan

for f in T.frame.unique():
    t = T.loc[T.frame == f]
    if len(t) > 1:
        for p in t.particle.unique():
            xp = t.loc[t.particle == p].x.values
            yp = t.loc[t.particle == p].y.values
            dist = []
            others = np.delete(t.particle.unique(), np.where(t.particle.unique() == p))
            for o in others:
                xo = t.loc[t.particle == o].x.values
                yo = t.loc[t.particle == o].y.values
                D = np.sqrt(np.abs(xp - xo)**2 + np.abs(yp - yo)**2)
                dist = np.append(dist, D)
            T.loc[T.loc[(T.frame == f) & (T.particle == p)].index, 'nnDist'] = np.min(dist)
            T.loc[T.loc[(T.frame == f) & (T.particle == p)].index, 'nnP'] = int(others[np.argmin(dist)])

In [10]:
T.tail()

,y,x,frame,particle,species,age,nnDist,nnP
109727,1208.843552,1548.434764,31489,549,aeg,21,1303.202926,278.0
109731,1208.774694,1548.276203,31490,549,aeg,21,1300.354592,278.0
109732,2541.681674,813.661191,31490,407,aeg,21,1521.939629,549.0
109730,712.296558,346.431153,31490,278,aeg,21,1300.354592,549.0
109733,2551.416768,2893.404085,31490,603,aeg,21,1900.541174,549.0
